In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()

In [7]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [8]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
correct value between  : 6.6 1.2
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
---------

In [9]:
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [10]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.610382,1.705521,0.388436,0.140427,2.136970,10.125990,12.254115,31.905469,42.709201,21.534201,0.200625,73.229167,2.769089,1.843012,156.329271,158.482795,2.297240,1.712760
std,6.062216,0.100203,0.323505,0.104645,2.281611,6.187555,7.571422,18.703486,26.222292,16.576035,2.045443,13.361351,1.809743,0.163008,95.745881,94.697432,1.065408,1.062683
min,6.700000,0.000000,0.080000,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,30.000000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [11]:
all_train_data3=pd.DataFrame()
for i in range(all_train_data1.shape[1]):
    all_train_data3[i]=(all_train_data1[i] - all_train_data1[i].mean()) / all_train_data1[i].std()
all_train_data3[18]=all_train_data1[9]

In [12]:
all_train_data3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5760.000000
mean,1.396637e-14,-5.107358e-13,-1.330514e-14,5.078230e-15,2.771201e-15,-2.338465e-15,-3.474295e-15,-4.431640e-16,2.485088e-16,-2.134674e-16,4.130921e-15,-9.199431e-16,9.995284e-16,3.404591e-13,8.574353e-17,5.245418e-16,5.851993e-16,1.040902e-15,21.534201
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,16.576035
min,-2.624516e+00,-1.702060e+01,-9.534177e-01,-1.341940e+00,-9.366061e-01,-1.636509e+00,-1.446771e+00,-1.705857e+00,-1.628736e+00,-1.299117e+00,-9.808389e-02,-3.310232e+00,-1.530101e+00,-1.130626e+01,-1.631707e+00,-1.673570e+00,-2.156207e+00,-1.611732e+00,0.000000
25%,-7.605110e-01,-1.053067e+00,-4.279241e-01,-6.730107e-01,-4.544905e-01,-6.829821e-01,-6.543176e-01,-7.434694e-01,-7.134846e-01,-7.561640e-01,-9.808389e-02,-6.907360e-01,-6.459971e-01,-8.773311e-01,-8.807613e-01,-8.710141e-01,-7.482952e-01,-7.648192e-01,9.000000
50%,6.426991e-02,-5.509630e-02,-1.497217e-01,-2.907653e-01,-2.353471e-01,-2.466224e-01,-2.977135e-01,-1.553437e-01,-1.795877e-01,-2.132115e-01,-9.808389e-02,1.325340e-01,-2.592018e-01,-2.638650e-01,-3.898786e-01,-3.905364e-01,-1.851306e-01,-2.002106e-01,18.000000
75%,7.240946e-01,9.428745e-01,1.903036e-01,3.781641e-01,7.145369e-02,4.644824e-01,3.626644e-01,5.397139e-01,5.449866e-01,5.107252e-01,-9.808389e-02,8.061186e-01,3.486194e-01,3.496011e-01,5.918869e-01,6.390586e-01,5.657555e-01,5.526008e-01,30.000000
max,2.208700e+00,2.938816e+00,2.219921e+01,1.108104e+01,1.265029e+01,5.797768e+00,7.758898e+00,1.064478e+01,5.273788e+00,5.457626e+00,3.607990e+01,1.928760e+00,1.062632e+01,7.097729e+00,2.127201e+00,2.128011e+00,5.071072e+00,4.975368e+00,112.000000


In [13]:
all_train_data3.shape

(5760, 19)

In [14]:
year_data3={}
for m in range(12):
    month_data3=np.empty((18,480))
    month_data3[:,:]=all_train_data3.T.iloc[:18,m*480:(m+1)*480]
    year_data3[m]=month_data3

In [15]:
x=np.empty((12*471,18*9))
y=np.empty((12*471,1))

In [16]:
year_data3[0].shape

(18, 480)

In [17]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=year_data3[m][:,g:g+9].reshape(1,-1)
        y[m*471+g:m*471+g+1,:]=all_train_data3.T.iloc[18,m*480+9+g]

In [18]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,cross_val_score

In [19]:
kf=KFold(n_splits=25,shuffle=True,random_state=7).split(x)
Lasso=linear_model.Lasso(alpha=0.00005)
score=-cross_val_score(Lasso,x,y,scoring='neg_mean_squared_error',cv=kf)
print('score : ', score)
print('mean score : score',np.mean(score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

score 30.88794189964675


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [20]:
np.sqrt(30.88794189964675)

5.5576921378974165

In [21]:
import torch 
from torch import optim,nn

In [22]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [23]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [24]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [25]:
x1=torch.Tensor(x)
y1=torch.Tensor(y)

In [27]:
def get_net():
    return nn.Sequential(nn.Linear(x1.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,10000,0.00001,5,x1,y1,256,0,0)

epoch = 1 train_loss : 730.267761 , test loss : 732.536072
epoch = 2 train_loss : 728.846313 , test loss : 731.079163
epoch = 3 train_loss : 727.424377 , test loss : 729.622131
epoch = 4 train_loss : 726.028992 , test loss : 728.188721
epoch = 5 train_loss : 724.625916 , test loss : 726.748291
epoch = 6 train_loss : 723.233887 , test loss : 725.321716
epoch = 7 train_loss : 721.844666 , test loss : 723.898804
epoch = 8 train_loss : 720.488464 , test loss : 722.510559
epoch = 9 train_loss : 719.103760 , test loss : 721.090515
epoch = 10 train_loss : 717.741211 , test loss : 719.693787
epoch = 11 train_loss : 716.380127 , test loss : 718.299561
epoch = 12 train_loss : 715.014343 , test loss : 716.901123
epoch = 13 train_loss : 713.670593 , test loss : 715.525391
epoch = 14 train_loss : 712.312561 , test loss : 714.138000
epoch = 15 train_loss : 710.973022 , test loss : 712.767395
epoch = 16 train_loss : 709.594727 , test loss : 711.357422
epoch = 17 train_loss : 708.249268 , test loss : 

epoch = 139 train_loss : 583.551514 , test loss : 585.777283
epoch = 140 train_loss : 582.999146 , test loss : 585.258179
epoch = 141 train_loss : 582.482544 , test loss : 584.790344
epoch = 142 train_loss : 581.956482 , test loss : 584.316589
epoch = 143 train_loss : 581.437012 , test loss : 583.842773
epoch = 144 train_loss : 580.920166 , test loss : 583.364746
epoch = 145 train_loss : 580.421204 , test loss : 582.916443
epoch = 146 train_loss : 579.904236 , test loss : 582.450256
epoch = 147 train_loss : 579.430664 , test loss : 582.022583
epoch = 148 train_loss : 578.935913 , test loss : 581.582947
epoch = 149 train_loss : 578.446777 , test loss : 581.145081
epoch = 150 train_loss : 577.954163 , test loss : 580.696411
epoch = 151 train_loss : 577.454773 , test loss : 580.245178
epoch = 152 train_loss : 576.990845 , test loss : 579.833740
epoch = 153 train_loss : 576.530334 , test loss : 579.425354
epoch = 154 train_loss : 576.050293 , test loss : 578.997131
epoch = 155 train_loss :

epoch = 276 train_loss : 534.111816 , test loss : 543.818726
epoch = 277 train_loss : 533.830872 , test loss : 543.593323
epoch = 278 train_loss : 533.551086 , test loss : 543.371521
epoch = 279 train_loss : 533.268005 , test loss : 543.137634
epoch = 280 train_loss : 532.989197 , test loss : 542.909912
epoch = 281 train_loss : 532.709473 , test loss : 542.679443
epoch = 282 train_loss : 532.433777 , test loss : 542.445923
epoch = 283 train_loss : 532.161621 , test loss : 542.225464
epoch = 284 train_loss : 531.881775 , test loss : 541.998352
epoch = 285 train_loss : 531.605591 , test loss : 541.768982
epoch = 286 train_loss : 531.331116 , test loss : 541.543884
epoch = 287 train_loss : 531.063232 , test loss : 541.332153
epoch = 288 train_loss : 530.787292 , test loss : 541.104309
epoch = 289 train_loss : 530.504944 , test loss : 540.874023
epoch = 290 train_loss : 530.244568 , test loss : 540.663818
epoch = 291 train_loss : 529.965881 , test loss : 540.429016
epoch = 292 train_loss :

epoch = 414 train_loss : 498.468109 , test loss : 513.644348
epoch = 415 train_loss : 498.220032 , test loss : 513.411255
epoch = 416 train_loss : 497.971802 , test loss : 513.193542
epoch = 417 train_loss : 497.722565 , test loss : 512.952637
epoch = 418 train_loss : 497.471619 , test loss : 512.739075
epoch = 419 train_loss : 497.219513 , test loss : 512.519592
epoch = 420 train_loss : 496.971771 , test loss : 512.287964
epoch = 421 train_loss : 496.722382 , test loss : 512.066711
epoch = 422 train_loss : 496.474579 , test loss : 511.842957
epoch = 423 train_loss : 496.224854 , test loss : 511.606628
epoch = 424 train_loss : 495.972046 , test loss : 511.392120
epoch = 425 train_loss : 495.719360 , test loss : 511.181854
epoch = 426 train_loss : 495.470215 , test loss : 510.938141
epoch = 427 train_loss : 495.220215 , test loss : 510.727692
epoch = 428 train_loss : 494.968994 , test loss : 510.495239
epoch = 429 train_loss : 494.718323 , test loss : 510.281616
epoch = 430 train_loss :

epoch = 552 train_loss : 462.884949 , test loss : 480.321564
epoch = 553 train_loss : 462.614075 , test loss : 480.059906
epoch = 554 train_loss : 462.341980 , test loss : 479.801910
epoch = 555 train_loss : 462.072266 , test loss : 479.547607
epoch = 556 train_loss : 461.801300 , test loss : 479.268311
epoch = 557 train_loss : 461.531494 , test loss : 478.996674
epoch = 558 train_loss : 461.258362 , test loss : 478.741608
epoch = 559 train_loss : 460.986786 , test loss : 478.463348
epoch = 560 train_loss : 460.722046 , test loss : 478.210541
epoch = 561 train_loss : 460.448486 , test loss : 477.950714
epoch = 562 train_loss : 460.174652 , test loss : 477.658783
epoch = 563 train_loss : 459.904938 , test loss : 477.398956
epoch = 564 train_loss : 459.630371 , test loss : 477.153931
epoch = 565 train_loss : 459.359924 , test loss : 476.881226
epoch = 566 train_loss : 459.088135 , test loss : 476.617004
epoch = 567 train_loss : 458.814392 , test loss : 476.348267
epoch = 568 train_loss :

...

epoch = 2516 train_loss : 32.511421 , test loss : 31.775631
epoch = 2517 train_loss : 32.507694 , test loss : 31.773071
epoch = 2518 train_loss : 32.504196 , test loss : 31.768587
epoch = 2519 train_loss : 32.500130 , test loss : 31.764658
epoch = 2520 train_loss : 32.496944 , test loss : 31.763395
epoch = 2521 train_loss : 32.492119 , test loss : 31.756714
epoch = 2522 train_loss : 32.488304 , test loss : 31.744715
epoch = 2524 train_loss : 32.481998 , test loss : 31.731546
epoch = 2527 train_loss : 32.469013 , test loss : 31.730671
epoch = 2528 train_loss : 32.465858 , test loss : 31.725548
epoch = 2529 train_loss : 32.461197 , test loss : 31.723831
epoch = 2530 train_loss : 32.457951 , test loss : 31.723154
epoch = 2531 train_loss : 32.453892 , test loss : 31.716206
epoch = 2534 train_loss : 32.442249 , test loss : 31.709652
epoch = 2535 train_loss : 32.440086 , test loss : 31.707129
epoch = 2536 train_loss : 32.434933 , test loss : 31.703798
epoch = 2537 train_loss : 32.430672 , te

epoch = 2759 train_loss : 31.718941 , test loss : 31.094658
epoch = 2760 train_loss : 31.715797 , test loss : 31.086288
epoch = 2761 train_loss : 31.713238 , test loss : 31.082439
epoch = 2766 train_loss : 31.699909 , test loss : 31.081551
epoch = 2767 train_loss : 31.697092 , test loss : 31.076986
epoch = 2768 train_loss : 31.693909 , test loss : 31.074045
epoch = 2769 train_loss : 31.691378 , test loss : 31.072956
epoch = 2770 train_loss : 31.688847 , test loss : 31.063015
epoch = 2772 train_loss : 31.682983 , test loss : 31.052958
epoch = 2776 train_loss : 31.672600 , test loss : 31.043453
epoch = 2778 train_loss : 31.667940 , test loss : 31.039989
epoch = 2784 train_loss : 31.650675 , test loss : 31.039417
epoch = 2786 train_loss : 31.646048 , test loss : 31.018572
epoch = 2788 train_loss : 31.640266 , test loss : 31.018263
epoch = 2793 train_loss : 31.627319 , test loss : 31.018017
epoch = 2794 train_loss : 31.626125 , test loss : 31.007942
epoch = 2795 train_loss : 31.621407 , te

epoch = 3090 train_loss : 30.969393 , test loss : 30.484095
epoch = 3096 train_loss : 30.958271 , test loss : 30.479683
epoch = 3098 train_loss : 30.955101 , test loss : 30.475792
epoch = 3103 train_loss : 30.945875 , test loss : 30.463213
epoch = 3112 train_loss : 30.929394 , test loss : 30.460062
epoch = 3113 train_loss : 30.927164 , test loss : 30.448341
epoch = 3118 train_loss : 30.918171 , test loss : 30.448286
epoch = 3122 train_loss : 30.910803 , test loss : 30.443079
epoch = 3123 train_loss : 30.909693 , test loss : 30.440512
epoch = 3124 train_loss : 30.907148 , test loss : 30.439491
epoch = 3126 train_loss : 30.904032 , test loss : 30.439053
epoch = 3129 train_loss : 30.898127 , test loss : 30.427679
epoch = 3136 train_loss : 30.885387 , test loss : 30.423616
epoch = 3139 train_loss : 30.881472 , test loss : 30.420658
epoch = 3141 train_loss : 30.876680 , test loss : 30.418505
epoch = 3142 train_loss : 30.874752 , test loss : 30.410673
epoch = 3144 train_loss : 30.871429 , te

epoch = 3550 train_loss : 30.300165 , test loss : 29.971609
epoch = 3551 train_loss : 30.299610 , test loss : 29.965780
epoch = 3558 train_loss : 30.291759 , test loss : 29.962976
epoch = 3563 train_loss : 30.285797 , test loss : 29.955894
epoch = 3565 train_loss : 30.283667 , test loss : 29.953905
epoch = 3568 train_loss : 30.280384 , test loss : 29.950148
epoch = 3573 train_loss : 30.275164 , test loss : 29.945757
epoch = 3576 train_loss : 30.272030 , test loss : 29.944916
epoch = 3580 train_loss : 30.267632 , test loss : 29.939005
epoch = 3585 train_loss : 30.262415 , test loss : 29.938993
epoch = 3586 train_loss : 30.261202 , test loss : 29.936367
epoch = 3592 train_loss : 30.254789 , test loss : 29.932587
epoch = 3599 train_loss : 30.247536 , test loss : 29.929070
epoch = 3600 train_loss : 30.246508 , test loss : 29.927137
epoch = 3601 train_loss : 30.245638 , test loss : 29.922508
epoch = 3609 train_loss : 30.237417 , test loss : 29.920547
epoch = 3611 train_loss : 30.235008 , te

epoch = 4571 train_loss : 29.590120 , test loss : 29.427105
epoch = 4575 train_loss : 29.589664 , test loss : 29.421778
epoch = 4594 train_loss : 29.580877 , test loss : 29.417910
epoch = 4604 train_loss : 29.577026 , test loss : 29.416422
epoch = 4608 train_loss : 29.575607 , test loss : 29.414291
epoch = 4622 train_loss : 29.570015 , test loss : 29.414259
epoch = 4628 train_loss : 29.568312 , test loss : 29.408424
epoch = 4630 train_loss : 29.567320 , test loss : 29.402483
epoch = 4631 train_loss : 29.567558 , test loss : 29.396112
epoch = 4674 train_loss : 29.550777 , test loss : 29.396011
epoch = 4675 train_loss : 29.551880 , test loss : 29.394878
epoch = 4684 train_loss : 29.547985 , test loss : 29.385931
epoch = 4698 train_loss : 29.542534 , test loss : 29.384974
epoch = 4702 train_loss : 29.541191 , test loss : 29.380529
epoch = 4716 train_loss : 29.536871 , test loss : 29.377251
epoch = 4722 train_loss : 29.534512 , test loss : 29.370747
epoch = 4726 train_loss : 29.533689 , te